함수호출

In [1]:
import pandas as pd
import time,datetime
import numpy as np
from pandas import *
from informs import portfolio
import class_informs as inform

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import math
import random as ran
from scipy.stats import truncnorm
#import seaborn as sbn

import time,datetime
import cmath

데이터 전처리

In [2]:
#timeseries data
all_data=pd.read_csv('data/Timeseries_data_SP500.csv')
date_list=list(set(list(all_data['DATE'])))
for i in range(len(date_list)):
    ak=date_list[i]
    date_list[i]=ak[6:]+str('-')+ak[:2]+'-'+ak[3:5]
date_list.sort()
all_data['DATE']=pd.to_datetime(all_data['DATE'])
all_data=all_data.rename(columns={'NAME':'name','DATE':'date','SEDOL':'sedol','SECTOR':'sector','BETA':'beta','ALPHA_SCORE':'as','BENCH_WEIGHT':'bw',"MCAP_Q":'mq'})
sedol_list=all_data['sedol'].unique().tolist()



dic_data = {k: v for k, v in all_data.groupby('date')}

using_dic = dic_data[pd.to_datetime(date_list[0])]



asset_list = []
for i in using_dic["sedol"]:
    asset_list.append(i)


dic_sedol_as = {using_dic["sedol"][i] : using_dic["as"][i] for i in using_dic.index}

dic_bench = {using_dic["sedol"][i] : using_dic["bw"][i] for i in using_dic.index}

dic_beta = {using_dic["sedol"][i] : using_dic["beta"][i] for i in using_dic.index}

dic_sector = {using_dic["sector"][i] : [] for i in using_dic.index }
dic_MCAP = {using_dic["mq"][i] : [] for i in using_dic.index}

for i in using_dic.index:
    dic_sector[using_dic["sector"][i]].append(using_dic["sedol"][i])
    dic_MCAP[using_dic["mq"][i]].append(using_dic["sedol"][i])


#risk:cov_mat
def risk(date_str):
    risk_data=pd.read_csv('data/Riskmodels/cov_mat_%s.csv'%(date_str))
    risk_sedol=risk_data['ROW_INDEX'].unique().tolist()
    risk_mat = np.zeros((len(risk_sedol),len(risk_sedol)))
    risk_mat[np.triu_indices(len(risk_sedol), 0)] = list(risk_data['VALUE'])
    irows,icols = np.triu_indices(len(risk_sedol),0)
    risk_mat[icols,irows]=risk_mat[irows,icols]
    return risk_data,risk_sedol,risk_mat
risk_data,risk_sedol,risk_mat=risk(date_list[0])



함수 정의

In [3]:
as_ = dic_data[Timestamp('2007-01-03 00:00:00')]['as']
nbasset=len(risk_sedol)

def cal_obj(sol, risk_mat, as_) :
    a = np.full((len(risk_mat[0]), len(risk_mat[0])), 5000)
    risk_mat = risk_mat * a
    s_rr = np.dot(sol, risk_mat)
    s_r = np.dot(s_rr, sol)
    s_a = np.dot(sol, as_)
    obj = s_r+s_a
    return obj



def select_candi(sol) :
    
    save_sub = [0]*len(as_)
    save_add = [0]*len(as_)
    diff_sub = [0]*len(as_)
    diff_add = [0]*len(as_)
    candi = []
    
    for i in range(len(sol)) :
        if(sol[i] > 0) :
            w = sol[i]
            save_sub[i] = w
            diff_sub[i] = cal_obj(save_sub, risk_mat, alpha)
            np_su = np.array(diff_sub)
            
            if min(np_su[np_su>0]) == diff_sub[i] :
                for j in range(len(sol)) :
                    if sol[j] == 0 :
                        save_add[j] = w
                        diff_add[j] = cal_obj(save_add, risk_mat, alpha)
                        save_add[j] = 0
            save_sub[i] = 0

    np_ad = np.array(diff_add)

    if sol[diff_sub.index((min(np_su[np_su>0])))] == sol[diff_add.index((min(np_ad[np_ad>0])))] :
        sol[diff_add.index((min(np_ad[np_ad>min(np_ad[np_ad>0])])))] = 0.1
    else :
        sol[diff_add.index((min(np_ad[np_ad>0])))] = 0.1
        
    sol[diff_sub.index((min(np_su[np_su>0])))] = 0

    for val in range(len(sol)) :
        if sol[val] > 0. :
            candi.append(val)
            
    return candi



def update(sol,lim_time) :
    start_time = time.time()
    ex = inform.informs(10000)
    ex.set_omega(risk_mat)

    new_list = []          
    for k in risk_sedol :
        new_list.append(sol[k])

    result_list = []

    while(True):

        if time.time() - start_time > lim_time:
            break
        con = select_candi(new_list)
        ex.set_con(cons=con)
        new_list2 = []
        new_list3 = []
        try:
            aabbaa = ex.solve(0.01)

            if aabbaa != 0:
                aaaa = aabbaa[0]
                bbbb= aabbaa[1]
                for k in risk_sedol :
                    new_list2.append(aaaa[k])
                for k in risk_sedol :
                    new_list3.append(bbbb[k])
                new_list = new_list2

                result_list.append(aabbaa[2])

            else:
                print("TE infeasible")
        except:
            print("no solution")


    return result_list


def sample_Z(m, n):
    return Variable(torch.Tensor(np.random.uniform(0., 1., size=[m, n])))


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return np.random.normal(size=size, scale=xavier_stddev)


class Generator(nn.Module):
    def __init__(self,  in_dim=10, hd_dim=50, out_dim=nbasset):
        super(Generator, self).__init__()
        
        self.G_W1 = nn.Parameter(torch.from_numpy(xavier_init([in_dim, hd_dim])).float())
        self.G_b1 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
    
        self.G_W12 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, hd_dim])).float())
        self.G_b12 = nn.Parameter(torch.from_numpy(np.zeros(shape=[hd_dim])).float())
        
        
        self.G_W2 = nn.Parameter(torch.from_numpy(xavier_init([hd_dim, out_dim])).float())
        self.G_b2 = nn.Parameter(torch.from_numpy(np.zeros(shape=[out_dim])).float())

    def forward(self, x):
        x = F.relu( x @ self.G_W1 + self.G_b1)
        x = x @ self.G_W12 + self.G_b12
        x2 = F.relu( x @ self.G_W12 + self.G_b12)
        x2 = x @ self.G_W2 + self.G_b2
        return F.sigmoid(x2*0.1)


class Discriminator(nn.Module):
    def __init__(self, in_dim=nbasset,train=False):
        super(Discriminator, self).__init__()
        self.tot_infs = 0.0
        self.tot_infs_nor = 0.0
        self.train = train
        self.x_threshold = 0.001
        self.inf_scale = 100.0
        self.Zstar = 100000
        
        self.in_dim = in_dim

        np.random.seed(0)
        self.Omega = Variable(torch.from_numpy(risk_mat)).float()

        self.card_upper = 70.
        self.card_lower = 60.
        
        
    def forward(self, x):
        tot_infs = Variable(torch.zeros(x.size()[0]))
        x_bw = Variable(torch.from_numpy(bw).float())
        alp = Variable(torch.Tensor(alpha).float())   
        bet = Variable(torch.Tensor(beta).float())
 
        # (4)
        tot_infs += F.relu(1. - torch.sum(x,1))
        tot_infs += F.relu(torch.sum(x,1) - 1.)
        
        # (5)
        tot_infs += F.relu(torch.abs(x-x_bw)@ Variable(torch.ones(self.in_dim))-0.05)
        

        # (6)
        for j in range(nbsector):
            k= (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_sector[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (7)
        for j in range(nbmq):
            k = (x-x_bw) @ Variable(
                torch.Tensor([1 if i in group_by_mq[j] else 0 for i in range(self.in_dim)]).float())  #sum
            l = torch.abs(k)
            tot_infs += F.relu(l - 0.1)
            
        # (8)
        k = (x - x_bw)@bet
        l = torch.abs(k)
        tot_infs += F.relu(l - 0.1)
        
        # (9) lb <= card(x) <= b
        if self.train:
            num_non_zeros = torch.sum(torch.tanh(x / self.x_threshold), 1)
            tot_infs += F.relu(num_non_zeros - self.card_upper) + F.relu(self.card_lower - num_non_zeros)
        else:
            num_non_zeros = torch.sum(x.data>self.x_threshold, 1).float()
            tot_infs += Variable(np.maximum(num_non_zeros - self.card_upper, 0) + np.maximum(self.card_lower - num_non_zeros, 0))
    
        
        # (10)
        l = 0.5*torch.sum(torch.abs(x - x_bw))
        tot_infs += F.relu(0.6 - l)
    
        
        # (11) a<=dQd <= b
        dQ = (x-x_bw) @ self.Omega
        size_x = x.size()
        dQd = torch.bmm(dQ.view(size_x[0],1,size_x[1]), (x-x_bw).view(size_x[0],size_x[1],1)).squeeze(1).squeeze(1)
        tot_infs += F.relu(dQd - 0.01)
        tot_infs += F.relu(0.0025 - dQd)
        
        
        # min dQd + ad (<= Zstar)
        l = (x-x_bw)@alp
        tot_infs += (F.relu(100.*dQd - 100.*l - self.Zstar))
        #print((F.relu(100.*dQd - l - self.Zstar)))
        self.objs = 100.*dQd - 100.*l

        fea_probs = F.relu(1. - F.tanh(tot_infs/self.inf_scale))
        
        return fea_probs
        

def GAN(rotation):
    X_dim = nbasset
    HL_dim = 50
    Z_dim = 10

    G = Generator(Z_dim, HL_dim, X_dim)

    sample_D = Discriminator(X_dim, train=False)
    train_D = Discriminator(X_dim, train=True)

    G_solver = optim.Adam(G.parameters(), lr=1e-3)


    mb_size = 128

    best_obj = 100000.0
    best_sols = []

    losses = []

    start_time = time.time()

    feasible_threshold = 0.95

    adaptive_inf_scale = float(nbasset)

    for it in range(rotation):
        if it % 1000 == 0:
            sample_noise = sample_Z(128, Z_dim)
            samples = G(sample_noise)

            sample_D.Zstar = best_obj
            sample_D.inf_scale = adaptive_inf_scale

            sample_D_values = sample_D(samples).data.numpy()
            sample_obj_values = sample_D.objs.data.numpy()

            fea_sample_obj_values = sample_obj_values[sample_D_values > feasible_threshold]
            sols = samples.data.numpy()[sample_D_values > feasible_threshold]

            if len(fea_sample_obj_values > 0):
                best_idx = np.argmin(fea_sample_obj_values)
                if np.min(fea_sample_obj_values) < best_obj:
                    best_obj = np.minimum(fea_sample_obj_values[best_idx], best_obj)
                    best_sols.append(sols[best_idx])

        z = Variable(torch.randn(mb_size, Z_dim))

        train_D.Zstar = best_obj
        train_D.inf_scale = adaptive_inf_scale

        G_loss = -torch.mean(torch.log(train_D(G(z))))

        G.zero_grad()
        G_loss.backward()
        G_solver.step()
        
    return best_sols, sols,fea_sample_obj_values

def GAN_reulst(best_sols, sols,fea_sample_obj_values):    
    card_list=[list(np.where(sols[i]>0.001)[0]) for i in range(len(sols))]

    card_freq=np.zeros(nbasset)
    for i in range(len(card_list)):
        for j in range(len(card_list[i])):
            card_freq[card_list[j]]+=1
            
    best=np.argsort(sum(sols))[::-1][:len(card_freq[card_freq>0])]

    bestsol=np.where(best_sols[0]>0.001)[0]
    return best,bestsol

def initial_card(init_set,c_time=5.,gan_rot=3000.):
    if init_set=='cplex':
        sedol_var_list =[]
        for i in risk_sedol:
            sedol_var_list.append("d"+str(i))

        for i in risk_sedol:
            sedol_var_list.append("q"+str(i))

        sedol_var_list.append("assum")
        global alpha
        alpha = []
        for i in risk_sedol:
            alpha.append(-10000.*dic_sedol_as[i])

        qmat=[]
        for i in range(len(risk_mat)):
            qmat_1=[]
            qmat_1.append(sedol_var_list)
            new_risk_mat=[]
            for j in risk_mat[i]:
                new_risk_mat.append(10000*j)
            for j in risk_mat[i]:
                new_risk_mat.append(0)
            new_risk_mat.append(0)
            qmat_1.append(new_risk_mat)
            qmat.append(qmat_1)

        for i in range(len(risk_mat)):
            qmat_1=[]
            qmat_1.append(sedol_var_list)
            new_risk_mat=[]
            for j in risk_mat[i]:
                new_risk_mat.append(0)
            for j in risk_mat[i]:
                new_risk_mat.append(0)
            new_risk_mat.append(0)
            qmat_1.append(new_risk_mat)
            qmat.append(qmat_1)

        for i in range(1):
            qmat_1=[]
            qmat_1.append(sedol_var_list)
            new_risk_mat=[]
            for j in risk_mat[i]:
                new_risk_mat.append(0)
            for j in risk_mat[i]:
                new_risk_mat.append(0)
            new_risk_mat.append(0)
            qmat_1.append(new_risk_mat)
            qmat.append(qmat_1)

        q_con1 = []
        q_con2 = []
        q_val = []


        for i in range(len((risk_mat[0]))):
            for j in range(len(risk_mat[0])):
                if j >= i:
                    q_con1.append(i)
                    q_con2.append(j)
                    if i == j:
                        ex_list = list(risk_mat[i])
                        q_val.append(0.5*ex_list[j]*10000)
                    else:
                        ex_list = list(risk_mat[i])
                        q_val.append(ex_list[j]*10000)

        Q_con = []
        Q_con.append(q_con1)
        Q_con.append(q_con2)
        Q_con.append(q_val)

        sol = portfolio(sector=dic_sector,bench=dic_bench,asset=risk_sedol,MCAPQ=dic_MCAP,beta=dic_beta,alpha=alpha,qmat=qmat, Q_con=Q_con,multiple=10000,time_init=c_time)[0]

    elif init_set=='GAN':

        using_dic['index']=list(using_dic.index)


        bw=list(using_dic['bw'])
        bw=np.array(bw)


        beta=list(using_dic['beta'])
        beta=np.array(beta)

        alpha=list(using_dic['as'])
        alpha=np.array(alpha)

        sector_set = set(list(using_dic['sector']))
        sector_list = list(using_dic['sector'])

        gbs=using_dic[['sector', 'index']].apply(tuple, axis=1)
        group_by_sector = [[y[1] for y in gbs if y[0]==x] for x in sector_set]

        mq_list=list(using_dic['mq'])
        mq_set=set(list(using_dic['mq']))

        gbm=using_dic[['mq', 'index']].apply(tuple, axis=1)
        group_by_mq = [[y[1] for y in gbm if y[0]==x] for x in mq_set]

        nbasset=len(bw)
        nbsector=len(sector_set)
        nbmq=len(mq_set)

        best_sols, sols,fea_sample_obj_values=GAN(gan_rot)
        best,bestsol=GAN_reulst(best_sols, sols,fea_sample_obj_values)

        ###code for best_sols    
        #     sol={}
        #     for i in range(len(risk_sedol)):
        #         sol[risk_sedol[i]]=best_sols[0][i] 

        print(len(best))


        sol={}
        for i in range(len(risk_sedol)):
            if i in best: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.

    elif init_set=='random':
        r=list(set(list(np.random.randint(len(risk_sedol),size=70))))
        sol={}
        for i in range(len(risk_sedol)):
            if i in r: sol[risk_sedol[i]]=1.
            else: sol[risk_sedol[i]]=0.
    return sol
    


solve!

In [4]:
#initial set by GAN, cplex, 
init_set='cplex'

sol = initial_card(init_set,c_time=15.,gan_rot=3000)
result = update(sol,100.)

print("------objective value 변화------")
print(result)


CPXPARAM_TimeLimit                               15
CPXPARAM_Read_DataCheck                          1
CPXPARAM_Threads                                 1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0
Tried aggregator 2 times.
MIQCP Presolve eliminated 1486 rows and 2 columns.
MIQCP Presolve modified 1476 coefficients.
Aggregator did 58 substitutions.
Reduced MIQCP has 1941 rows, 2839 columns, and 197656 nonzeros.
Reduced MIQCP has 492 binaries, 0 generals, 0 SOSs, and 0 indicators.
Reduced MIQCP has 2 quadratic constraints.
Presolve time = 0.17 sec. (431.54 ticks)
Probing time = 0.04 sec. (21.83 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.04 sec. (39.21 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

      0  